In [1]:
import os
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject

# ------------------------------------------------------------------
# 1) Paths & parameters (your existing code)
# ------------------------------------------------------------------
dem_path = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/DEM_characteristics_300m/dem_filledfiltered.tif"

terrain_folder = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/DEM_characteristics_300m"
bioclim_folder = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/bioclimatic32733/"

categorical_rasters = {
    "labelled_ecosystems32733": "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/labelled_ecosystems32733.tif",
    "landsurfaceforms":          "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/landsurfaceforms/landsurfaceforms.tif",
    "lithology_raster":          "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/lithology_rasterized.tif",
    "soil_raster":               "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/angola_soil_data_raster.tif",
}

def list_tifs(folder):
    return sorted([
        os.path.join(folder,f) for f in os.listdir(folder)
        if f.lower().endswith(".tif") and not f.startswith("._")
    ])

covariate_paths = (
    list_tifs(terrain_folder) +
    list_tifs(bioclim_folder) +
    list(categorical_rasters.values())
)

# ------------------------------------------------------------------
# 2) Helper to resample to 1 km
# ------------------------------------------------------------------
def resample_to_1km(src_path, dst_path, is_categorical=False):
    with rasterio.open(src_path) as src:
        # target resolution (in map units, e.g. meters)
        target_res = 1000  

        transform, width, height = calculate_default_transform(
            src.crs, src.crs,
            src.width, src.height,
            *src.bounds,
            resolution=target_res
        )

        kwargs = src.meta.copy()
        kwargs.update({
            "transform": transform,
            "width": width,
            "height": height
        })

        resampling = Resampling.nearest if is_categorical else Resampling.cubic

        with rasterio.open(dst_path, "w", **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=src.crs,
                    resampling=resampling
                )

# ------------------------------------------------------------------
# 3) Check resolution and resample if ~300 m
# ------------------------------------------------------------------
for tif in covariate_paths:
    with rasterio.open(tif) as src:
        xres, yres = src.res
        print(f"{os.path.basename(tif)}  ->  {xres:.1f} m x {yres:.1f} m")

        # flag ~300 m resolution (tolerate small rounding)
        if abs(xres - 300) < 10 and abs(yres - 300) < 10:
            # determine output path
            out_path = tif.replace(".tif", "_1km.tif")
            # check if it's categorical
            is_cat = tif in categorical_rasters.values()
            print(f"   Resampling to 1 km ({'categorical' if is_cat else 'continuous'}) -> {out_path}")
            resample_to_1km(tif, out_path, is_categorical=is_cat)

MRVBF.tif  ->  300.0 m x 300.0 m
   Resampling to 1 km (continuous) -> /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/DEM_characteristics_300m/MRVBF_1km.tif
RLD.tif  ->  300.0 m x 300.0 m
   Resampling to 1 km (continuous) -> /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/DEM_characteristics_300m/RLD_1km.tif
aspect.tif  ->  300.0 m x 300.0 m
   Resampling to 1 km (continuous) -> /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/DEM_characteristics_300m/aspect_1km.tif
aspect_cos.tif  ->  300.0 m x 300.0 m
   Resampling to 1 km (continuous) -> /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/DEM_characteristics_300m/aspect_cos_1km.tif
aspect_sin.tif  ->  300.0 m x 300.0 m
   Resampling to 1 km (continuous) -> /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/DEM_characteristics_300m/aspect_sin_1km.tif
dem_filledfiltered.tif  ->  300.0 m x 300.0 m
   Resampling to 1 km (continuous) -> /Volumes/One_Touch/ango

In [5]:
import os
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject

# ------------------------------
# Paths to rasters


# Categorical rasters
categorical_rasters = [
    "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/labelled_ecosystems32733.tif",
    "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/landsurfaceforms/landsurfaceforms.tif",
    "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/angola_soil_data_raster.tif",
]

# Target resolutions (meters)

target_res_categorical = 1000   # 1 km for ecosystems/soil


# ------------------------------
# Helper to list .tif files in folder
# ------------------------------
def list_tifs(folder):
    return sorted([
        os.path.join(folder, f)
        for f in os.listdir(folder)
        if f.lower().endswith(".tif") and not f.startswith("._")
    ])

# ------------------------------
# Function to resample raster
# ------------------------------
def resample_raster(src_path, dst_path, target_res, is_categorical=False):
    with rasterio.open(src_path) as src:
        print(f"\nProcessing {os.path.basename(src_path)}")
        print(f"Original resolution: {src.res[0]:.2f} x {src.res[1]:.2f} (units: {src.crs})")

        transform, width, height = calculate_default_transform(
            src.crs, src.crs,
            src.width, src.height,
            *src.bounds,
            resolution=target_res
        )

        kwargs = src.meta.copy()
        kwargs.update({
            "transform": transform,
            "width": width,
            "height": height
        })

        resampling_method = Resampling.nearest if is_categorical else Resampling.cubic

        with rasterio.open(dst_path, "w", **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=src.crs,
                    resampling=resampling_method
                )

        print(f"Resampled to ~{target_res} m -> {os.path.basename(dst_path)}")


# ------------------------------
# Resample categorical rasters
# ------------------------------
for path in categorical_rasters:
    out_path = path.replace(".tif", "_1km.tif")
    resample_raster(path, out_path, target_res_categorical, is_categorical=True)



Processing labelled_ecosystems32733.tif
Original resolution: 90.52 x 90.52 (units: EPSG:32733)
Resampled to ~1000 m -> labelled_ecosystems32733_1km.tif

Processing landsurfaceforms.tif
Original resolution: 91.78 x 91.77 (units: EPSG:32733)
Resampled to ~1000 m -> landsurfaceforms_1km.tif

Processing angola_soil_data_raster.tif
Original resolution: 30.59 x 30.59 (units: EPSG:32733)
Resampled to ~1000 m -> angola_soil_data_raster_1km.tif
